In [33]:
# -*- coding: utf-8 -*-
"""
Segmentación Fondo / Flóculos (blanco) / Filamentos + Excel (sin métricas por mL).
Robusto a inversión: usa flatten de fondo + selección automática de umbral según RA.
Analiza la imagen ya erosionada en disco.

Salidas:
- ..._overlay.png
- ..._masks.png  (gris / realce / mask_floc / mask_fil)
- ..._mask_bg.png, ..._mask_floc.png, ..._mask_fil.png
- ..._medidas.xlsx  (hojas: Resumen, Flocs_por_objeto, Pixeles_label, Config)
"""

from pathlib import Path
import cv2
import numpy as np
import math

# ========= CONFIGURA AQUÍ =========
ruta_base       = Path(r"C:\Users\PC\Desktop\indice de jenkins\procesadas\1_semana\procesadas_erosion")
nombre_imagen   = "1d.jpg"      # se analiza la 1d (ya erosionada)
RADIO_REL_CAMPO = 0.96          # círculo dentro del FOV

# Flóculos (blanco = masa)
BLUR_KSIZE      = 5             # suavizado previo
OPEN_CLOSE      = 3             # limpieza morfológica
BG_OPEN_K       = 81            # tamaño de apertura para aplanar fondo (ajústalo: 61–121, impar)
ADAPTIVE_BLOCK  = 51            # ventana para adaptiveThreshold (impar)
ADAPTIVE_C      = -3            # constante para adaptiveThreshold

# Selección automática por cobertura (RA)
RA_TARGET       = 0.60          # objetivo de cobertura de masa (aprox.)
RA_OK_MIN       = 0.20          # evita máscaras <20%
RA_OK_MAX       = 0.85          # evita máscaras >85%
PERC_SCAN       = (70, 75, 80, 85, 90)  # percentiles a probar si Otsu/adaptativo fallan

# Filamentos (realce + histeresis)
DARK_LINES      = False         # False=filamentos claros; True=oscuros
LINE_LENGTH     = 25            # 21–31 si son gruesos
LINE_WIDTH      = 3             # 3–5
ANGLES_DEG      = (0, 30, 60, 90, 120, 150)

PERC_HI         = 90            # umbral alto (percentil) para semillas
PERC_LO         = 75            # umbral bajo (percentil) para crecer
MIN_FIL_PIX     = 120           # descarta hebras muy chicas

SOLO_SOBRESAL   = True          # hebras que sobresalen de la masa
MARGEN_PX       = 6             # margen para excluir borde de masa
QUITAR_DE_MASA_MARGEN = 4       # restar filamentos a la masa

# ----- Escala (solo mm / mm², sin volúmenes) -----
PIXEL_MM        = 0.000637755102040816   # mm/px
DIAM_PX_CAMPO   = 2352                   # diámetro del campo (px)
MIN_FLOC_PIX    = 50                     # área mínima de flóculo (px)
# ==============================================


# ===== Utilidades =====
def _odd(n):  # hace impar
    n = int(max(3, n))
    return n if n % 2 == 1 else n + 1

def mascara_campo(gray, frac=0.96):
    h, w = gray.shape
    r = int(min(h, w)*frac/2)
    m = np.zeros_like(gray, np.uint8)
    cv2.circle(m, (w//2, h//2), r, 255, -1)
    return m

def limpiar(binimg, ksize):
    if not ksize or ksize < 2:
        return binimg
    k = np.ones((ksize, ksize), np.uint8)
    out = cv2.morphologyEx(binimg, cv2.MORPH_OPEN, k)
    out = cv2.morphologyEx(out,    cv2.MORPH_CLOSE, k)
    return out

def segmentar_floculos_estable(gray, mask_circle):
    """
    BLANCO = FLÓCULO.
    1) Aplana fondo por apertura grande (flat).
    2) Prueba Otsu sobre 'flat', adaptativo y percentiles.
    3) Elige el que deje RA en [RA_OK_MIN, RA_OK_MAX]; si varios, el más cercano a RA_TARGET.
    """
    h, w = gray.shape
    blur = cv2.GaussianBlur(gray, (_odd(BLUR_KSIZE), _odd(BLUR_KSIZE)), 0)

    # Flatten de fondo (white tophat)
    K = _odd(BG_OPEN_K)
    ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (K, K))
    bg  = cv2.morphologyEx(blur, cv2.MORPH_OPEN, ker)
    flat = cv2.subtract(blur, bg)  # resalta blanco local (masa)
    flat = cv2.normalize(flat, None, 0, 255, cv2.NORM_MINMAX)

    cand = []

    # A) Otsu sobre 'flat'
    _, m_otsu = cv2.threshold(flat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    m_otsu = cv2.bitwise_and(m_otsu, mask_circle)
    m_otsu = limpiar(m_otsu, OPEN_CLOSE)
    ra = np.count_nonzero(m_otsu) / max(1, np.count_nonzero(mask_circle))
    cand.append(("otsu_flat", m_otsu, ra))

    # B) Adaptive threshold sobre 'blur'
    m_ad = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                 cv2.THRESH_BINARY, _odd(ADAPTIVE_BLOCK), ADAPTIVE_C)
    m_ad = cv2.bitwise_and(m_ad, mask_circle)
    m_ad = limpiar(m_ad, OPEN_CLOSE)
    ra_ad = np.count_nonzero(m_ad) / max(1, np.count_nonzero(mask_circle))
    cand.append(("adaptive", m_ad, ra_ad))

    # C) Percentiles sobre 'flat'
    vals = flat[mask_circle > 0]
    for p in PERC_SCAN:
        thr = int(np.percentile(vals, p))
        _, m_p = cv2.threshold(flat, thr, 255, cv2.THRESH_BINARY)
        m_p = cv2.bitwise_and(m_p, mask_circle)
        m_p = limpiar(m_p, OPEN_CLOSE)
        ra_p = np.count_nonzero(m_p) / max(1, np.count_nonzero(mask_circle))
        cand.append((f"perc{p}", m_p, ra_p))

    # Selección por RA
    ok = [(name, m, ra) for (name, m, ra) in cand if RA_OK_MIN <= ra <= RA_OK_MAX]
    if ok:
        # el más cercano al objetivo
        name, best, best_ra = min(ok, key=lambda t: abs(t[2] - RA_TARGET))
        print(f"[Floc] elegido={name}, RA={best_ra:.3f}")
        return best
    # Si todos malos, dejar el que se acerque más a 0.5
    name, best, best_ra = min(cand, key=lambda t: abs(t[2] - 0.5))
    print(f"[Floc] fallback={name}, RA={best_ra:.3f}")
    return best

def kernel_lineal(length, width, angle_deg):
    L = max(5, int(length) | 1)
    W = max(1, int(width)  | 1)
    k = np.zeros((L, L), np.uint8)
    cv2.line(k, (L//2, (L-W)//2), (L//2, (L+W)//2), 1, W)
    M = cv2.getRotationMatrix2D((L/2, L/2), angle_deg, 1.0)
    k = cv2.warpAffine(k, M, (L, L), flags=cv2.INTER_NEAREST)
    k[k>0] = 1
    return k

def realce_filamentos(gray):
    bg = cv2.morphologyEx(gray, cv2.MORPH_OPEN,
                          cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (41, 41)))
    base = cv2.subtract(gray, bg)
    resp_max = np.zeros_like(base)
    for ang in ANGLES_DEG:
        k = kernel_lineal(LINE_LENGTH, LINE_WIDTH, ang)
        resp = cv2.morphologyEx(base, cv2.MORPH_BLACKHAT if DARK_LINES else cv2.MORPH_TOPHAT, k)
        resp_max = np.maximum(resp_max, resp)
    return cv2.normalize(resp_max, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def reconstruccion_por_histeresis(resp, region, perc_hi=90, perc_lo=75):
    vals = resp[region > 0]
    if vals.size == 0:
        return np.zeros_like(resp)
    thr_hi = int(np.percentile(vals, perc_hi))
    thr_lo = int(np.percentile(vals, perc_lo))
    _, seeds   = cv2.threshold(resp, thr_hi, 255, cv2.THRESH_BINARY)
    _, support = cv2.threshold(resp, thr_lo, 255, cv2.THRESH_BINARY)
    seeds   = cv2.bitwise_and(seeds,   region)
    support = cv2.bitwise_and(support, region)

    rec = seeds.copy()
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    while True:
        dil = cv2.dilate(rec, k)
        new = cv2.bitwise_and(dil, support)
        if np.array_equal(new, rec): break
        rec = new
    return rec

def limpiar_componentes(binimg, min_area):
    n, labels, stats, _ = cv2.connectedComponentsWithStats(binimg, connectivity=8)
    out = np.zeros_like(binimg)
    for i in range(1, n):
        if stats[i, cv2.CC_STAT_AREA] >= min_area:
            out[labels == i] = 255
    return out

def overlay_colores(bgr, mask_bg, mask_floc, mask_fil, alpha=0.6):
    color = np.zeros_like(bgr)
    color[mask_bg  > 0] = (200,150, 80)   # AZUL
    color[mask_floc> 0] = ( 80,220,120)   # VERDE
    color[mask_fil > 0] = (  0,165,255)   # NARANJO
    return cv2.addWeighted(color, alpha, bgr, 1-alpha, 0)


# ===== Métricas (sin normalizar por volumen) =====
def obtener_AT_AHD(mask_total):
    k = np.ones((2,2), np.uint8)
    mask_hd = cv2.morphologyEx(mask_total, cv2.MORPH_OPEN, k)
    mask_hd = cv2.morphologyEx(mask_hd, cv2.MORPH_OPEN, k)
    return mask_total, mask_hd

def contornos_validos(mask, min_area=MIN_FLOC_PIX):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    return [c for c in cnts if cv2.contourArea(c) >= min_area]

def medidas_por_floc(cnts, px_area_mm2, px_mm):
    res = []
    for i, c in enumerate(cnts, 1):
        area_px = cv2.contourArea(c)
        perim_px = cv2.arcLength(c, True)
        M = cv2.moments(c)
        if M['m00'] == 0: continue
        cx = M['m10']/M['m00']; cy = M['m01']/M['m00']
        pts = c.reshape(-1,2)
        d = np.sqrt((pts[:,0]-cx)**2 + (pts[:,1]-cy)**2)
        rmin_px = np.min(d) if d.size>0 else 0.0
        rmax_px = np.max(d) if d.size>0 else 0.0

        area_mm2 = area_px * px_area_mm2
        perim_mm = perim_px * px_mm
        rmin_mm  = rmin_px * px_mm
        rmax_mm  = rmax_px * px_mm

        AR = 4.0*(rmax_mm - rmin_mm)/(np.pi*rmin_mm) + 1.0 if rmin_mm>0 else np.nan
        RN = (perim_mm**2)/(4.0*np.pi*area_mm2) if area_mm2>0 else np.nan

        res.append(dict(
            id=i, area_px=area_px, area_mm2=area_mm2,
            perim_px=perim_px, perim_mm=perim_mm,
            rmin_mm=rmin_mm, rmax_mm=rmax_mm, AR=AR, RN=RN,
            cx_px=cx, cy_px=cy
        ))
    return res

def fractal_dimension_loglog(rows):
    xs, ys = [], []
    for r in rows:
        if r['area_mm2']>0 and r['perim_mm']>0:
            xs.append(np.log(r['perim_mm']))
            ys.append(np.log(r['area_mm2']))
    if len(xs) >= 2:
        m, _ = np.polyfit(xs, ys, 1)
        return float(m)
    return np.nan

def calc_fil_length_mm(mask_fil, px_mm):
    method = "area_over_width"
    try:
        from skimage.morphology import skeletonize
        skel = skeletonize((mask_fil>0).astype(bool))
        length_px = int(np.count_nonzero(skel))
        method = "skeletonize"
    except Exception:
        length_px = int(np.count_nonzero(mask_fil) / max(LINE_WIDTH,1))
    return length_px * px_mm, method


# --------- PROCESO (1 imagen) ---------
img_path = ruta_base / nombre_imagen
bgr  = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
if bgr is None: raise SystemExit(f"❌ No pude leer {img_path}")
gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

# Campo de visión
mask_circle = mascara_campo(gray, RADIO_REL_CAMPO)

# 1) Flóculos (BLANCO = masa) con segmentación estable
mask_floc_raw = segmentar_floculos_estable(gray, mask_circle)
mask_floc = mask_floc_raw.copy()

# 2) Filamentos: realce + histeresis
resp = realce_filamentos(gray)
region_fil = mask_circle.copy()
if SOLO_SOBRESAL:
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (_odd(2*MARGEN_PX+1), _odd(2*MARGEN_PX+1)))
    region_fil = cv2.bitwise_and(region_fil, cv2.bitwise_not(cv2.dilate(mask_floc, k)))
# Fallback si la región quedó casi vacía
if np.count_nonzero(region_fil) < 0.01 * np.count_nonzero(mask_circle):
    region_fil = mask_circle.copy()

mask_fil = reconstruccion_por_histeresis(resp, region_fil, PERC_HI, PERC_LO)
mask_fil = limpiar_componentes(mask_fil, MIN_FIL_PIX)

# 3) Restar filamentos a la masa (margen) para evitar solapes
if QUITAR_DE_MASA_MARGEN > 0:
    kd = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (_odd(2*QUITAR_DE_MASA_MARGEN+1), _odd(2*QUITAR_DE_MASA_MARGEN+1)))
    fil_dil = cv2.dilate(mask_fil, kd)
    mask_floc = cv2.bitwise_and(mask_floc, cv2.bitwise_not(fil_dil))

# 4) Fondo
mask_bg = mask_circle.copy()
mask_bg[mask_floc > 0] = 0
mask_bg[mask_fil  > 0] = 0

# ===== SALIDAS DE IMAGEN =====
overlay = overlay_colores(bgr, mask_bg, mask_floc, mask_fil, alpha=0.60)
out_overlay = img_path.with_name(f"{img_path.stem}_overlay.png");   cv2.imwrite(str(out_overlay), overlay)

h, w = gray.shape
tile_w, tile_h = w//2, h//2
to_bgr = lambda im: cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
rsz    = lambda im: cv2.resize(to_bgr(im), (tile_w, tile_h), interpolation=cv2.INTER_AREA)
panel  = np.vstack([ np.hstack([rsz(gray), rsz(resp)]),
                     np.hstack([rsz(mask_floc), rsz(mask_fil)]) ])
out_panel = img_path.with_name(f"{img_path.stem}_masks.png");       cv2.imwrite(str(out_panel), panel)

out_mask_bg   = img_path.with_name(f"{img_path.stem}_mask_bg.png");   cv2.imwrite(str(out_mask_bg),   mask_bg)
out_mask_floc = img_path.with_name(f"{img_path.stem}_mask_floc.png"); cv2.imwrite(str(out_mask_floc), mask_floc)
out_mask_fil  = img_path.with_name(f"{img_path.stem}_mask_fil.png");  cv2.imwrite(str(out_mask_fil),  mask_fil)

# ====== MÉTRICAS Y EXCEL (sin /mL) ======
px_mm       = float(PIXEL_MM)
px_area_mm2 = px_mm**2

pix_cir = int(np.count_nonzero(mask_circle > 0))
effective_area_mm2 = pix_cir * px_area_mm2

cnt_floc = int(np.count_nonzero((mask_floc>0)))
cnt_fil  = int(np.count_nonzero((mask_fil>0)))
cnt_bg   = int(np.count_nonzero((mask_bg>0)))
tot      = cnt_floc + cnt_fil + cnt_bg

RA_floc = 100.0*cnt_floc/tot if tot>0 else 0.0
RA_fil  = 100.0*cnt_fil /tot if tot>0 else 0.0
RA_bg   = 100.0*cnt_bg  /tot if tot>0 else 0.0

mask_AT, mask_AHD = obtener_AT_AHD(mask_floc_raw)
AT_mm2  = np.count_nonzero(mask_AT>0)  * px_area_mm2
AHD_mm2 = np.count_nonzero(mask_AHD>0) * px_area_mm2
AHD_sobre_AT = (AHD_mm2/AT_mm2) if AT_mm2>0 else np.nan

DeqT  = 2.0*math.sqrt(AT_mm2/np.pi)  if AT_mm2>0  else np.nan
DeqHD = 2.0*math.sqrt(AHD_mm2/np.pi) if AHD_mm2>0 else np.nan

cnts_total = contornos_validos(mask_AT, MIN_FLOC_PIX)
rows = medidas_por_floc(cnts_total, px_area_mm2, px_mm)
AR_mean = float(np.nanmean([r['AR'] for r in rows])) if rows else np.nan
RN_mean = float(np.nanmean([r['RN'] for r in rows])) if rows else np.nan
FD_val  = fractal_dimension_loglog(rows) if rows else np.nan

fil_length_mm_val, fil_len_method = calc_fil_length_mm(mask_fil, px_mm)

# ---- Exportar a Excel ----
try:
    import pandas as pd
except ImportError:
    raise SystemExit("Instala pandas y openpyxl:  pip install pandas openpyxl")

out_xlsx = img_path.with_name(f"{img_path.stem}_medidas.xlsx")

df_resumen = pd.DataFrame([{
    "Imagen": img_path.name,
    "Ancho (px)": w, "Alto (px)": h, "Diám. campo (px)": DIAM_PX_CAMPO,
    "mm/px": px_mm, "Área efectiva (mm²)": effective_area_mm2,
    "Fondo (px)": cnt_bg, "Flóculos (px)": cnt_floc, "Filamentos (px)": cnt_fil,
    "RA fondo (%)": RA_bg, "RA flóculos (%)": RA_floc, "RA filamentos (%)": RA_fil,
    "AT (mm²)": AT_mm2, "AHD (mm²)": AHD_mm2, "AHD/AT": AHD_sobre_AT,
    "DeqT (mm)": DeqT, "DeqHD (mm)": DeqHD,
    "AR̅": AR_mean, "RN̅": RN_mean, "FD": FD_val,
    "Long. filamentos (mm)": fil_length_mm_val
}])
df_resumen = df_resumen[[
    "Imagen","Ancho (px)","Alto (px)","Diám. campo (px)","mm/px","Área efectiva (mm²)",
    "Fondo (px)","Flóculos (px)","Filamentos (px)",
    "RA fondo (%)","RA flóculos (%)","RA filamentos (%)",
    "AT (mm²)","AHD (mm²)","AHD/AT","DeqT (mm)","DeqHD (mm)",
    "AR̅","RN̅","FD","Long. filamentos (mm)"
]]

df_flocs = pd.DataFrame(rows) if rows else pd.DataFrame(columns=[
    "id","area_px","area_mm2","perim_px","perim_mm","rmin_mm","rmax_mm","AR","RN","cx_px","cy_px"
])

tot_safe = max(1, tot)
df_pix = pd.DataFrame([{
    "imagen": img_path.name, "total_region": tot_safe, "bg_px": cnt_bg, "floc_px": cnt_floc, "fil_px": cnt_fil
}])

df_cfg = pd.DataFrame([{
    "PIXEL_MM": px_mm, "px_area_mm2": px_area_mm2,
    "DIAM_PX_CAMPO": DIAM_PX_CAMPO, "RADIO_REL_CAMPO": RADIO_REL_CAMPO,
    "EFFECTIVE_AREA_MM2": effective_area_mm2,
    "BG_OPEN_K": BG_OPEN_K, "ADAPTIVE_BLOCK": ADAPTIVE_BLOCK, "ADAPTIVE_C": ADAPTIVE_C,
    "RA_TARGET": RA_TARGET, "RA_OK_MIN": RA_OK_MIN, "RA_OK_MAX": RA_OK_MAX, "PERC_SCAN": str(PERC_SCAN),
    "SOLO_SOBRESAL": SOLO_SOBRESAL, "MARGEN_PX": MARGEN_PX, "QUITAR_DE_MASA_MARGEN": QUITAR_DE_MASA_MARGEN,
    "fil_length_method": fil_len_method,
    "DARK_LINES": DARK_LINES, "LINE_LENGTH": LINE_LENGTH, "LINE_WIDTH": LINE_WIDTH,
    "PERC_HI": PERC_HI, "PERC_LO": PERC_LO,
    "BLUR_KSIZE": BLUR_KSIZE, "OPEN_CLOSE": OPEN_CLOSE, "MIN_FLOC_PIX": MIN_FLOC_PIX,
    "ANGLES_DEG": str(ANGLES_DEG)
}])

with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_resumen.to_excel(writer, index=False, sheet_name="Resumen")
    df_flocs.to_excel(writer, index=False, sheet_name="Flocs_por_objeto")
    df_pix.to_excel(writer, index=False, sheet_name="Pixeles_label")
    df_cfg.to_excel(writer, index=False, sheet_name="Config")

print("✅ Listo")
print("Overlay:", out_overlay)
print("Panel  :", out_panel)
print("Masks -> BG:", out_mask_bg, " Floc:", out_mask_floc, " Fil:", out_mask_fil)
print("Excel  :", out_xlsx)



[Floc] elegido=adaptive, RA=0.332
✅ Listo
Overlay: C:\Users\PC\Desktop\indice de jenkins\procesadas\1_semana\procesadas_erosion\1d_overlay.png
Panel  : C:\Users\PC\Desktop\indice de jenkins\procesadas\1_semana\procesadas_erosion\1d_masks.png
Masks -> BG: C:\Users\PC\Desktop\indice de jenkins\procesadas\1_semana\procesadas_erosion\1d_mask_bg.png  Floc: C:\Users\PC\Desktop\indice de jenkins\procesadas\1_semana\procesadas_erosion\1d_mask_floc.png  Fil: C:\Users\PC\Desktop\indice de jenkins\procesadas\1_semana\procesadas_erosion\1d_mask_fil.png
Excel  : C:\Users\PC\Desktop\indice de jenkins\procesadas\1_semana\procesadas_erosion\1d_medidas.xlsx


In [11]:
# -*- coding: utf-8 -*-
"""
Segmentación Fondo / Flóculos (blanco) / Filamentos + Excel (sin métricas por mL).
Mejoras para filamentos:
- Realce multi-escala y multi-orientación
- Polaridad automática (claros u oscuros)
- Histeresis con auto-ajuste de percentiles para obtener RA_fil razonable
- Por defecto cuenta filamentos dentro y fuera de la masa

Salidas:
- ..._overlay.png
- ..._masks.png  (gris / realce / mask_floc / mask_fil)
- ..._mask_bg.png, ..._mask_floc.png, ..._mask_fil.png
- ..._medidas.xlsx  (Resumen, Flocs_por_objeto, Pixeles_label, Config)
"""

from pathlib import Path
import cv2
import numpy as np
import math

# ========= CONFIGURA AQUÍ =========
ruta_base       = Path(r"C:\Users\PC\Desktop\indice de jenkins\filtro erosion\2_semana\procesadas_erosion")
nombre_imagen   = "2a.jpg"      # se analiza la 1d (ya erosionada)
RADIO_REL_CAMPO = 0.96          # círculo dentro del FOV

# Flóculos (blanco = masa)
BLUR_KSIZE      = 5             # suavizado previo a Otsu
OPEN_CLOSE      = 3             # limpieza morfológica
BG_OPEN_K       = 81            # apertura grande para aplanar fondo (61–121, impar)

# Filamentos (realce + histeresis)
DARK_LINES_MODE = "auto"        # "auto" | "bright" (claros) | "dark" (oscuros)
LINE_LENGTHS    = [15, 25, 35]  # multi-escala (largo)
LINE_WIDTHS     = [2, 3, 4]     # multi-escala (ancho)
ANGLES_DEG      = [0, 22.5, 45, 67.5, 90, 112.5, 135, 157.5]

# Histeresis automática (busca RA_fil razonable en la región permitida)
AUTO_HYST       = True
HYST_HI_CANDS   = [97, 95, 92, 90, 88]   # percentiles altos candidatos
HYST_LO_DELTA   = 15                     # low = hi - delta (mín 60)
RA_FIL_TARGET   = 0.05                   # % objetivo en región (5%)
RA_FIL_OK_MIN   = 0.005                  # 0.5%   mínimo aceptable
RA_FIL_OK_MAX   = 0.15                   # 15%    máximo aceptable

MIN_FIL_PIX     = 80           # descarta hebras muy chicas (ajusta si salen puntitos)
SOLO_SOBRESAL   = False        # False: también dentro de la masa; True: solo los que sobresalen
MARGEN_PX       = 6            # margen para excluir borde de masa si SOLO_SOBRESAL=True
QUITAR_DE_MASA_MARGEN = 4      # restar filamentos a la masa para evitar solape verde/naranjo

# ----- Escala (mm / mm², sin volúmenes) -----
PIXEL_MM        = 0.000637755102040816   # mm/px
DIAM_PX_CAMPO   = 2352                   # diámetro del campo (px)
MIN_FLOC_PIX    = 50                     # área mínima de flóculo (px) para métricas morfológicas
# ==============================================


# ===== Utilidades =====
def _odd(n): n=int(max(3,n)); return n if n%2==1 else n+1

def mascara_campo(gray, frac=0.96):
    h, w = gray.shape
    r = int(min(h, w)*frac/2)
    m = np.zeros_like(gray, np.uint8)
    cv2.circle(m, (w//2, h//2), r, 255, -1)
    return m

def limpiar(binimg, ksize):
    if not ksize or ksize < 2: return binimg
    k = np.ones((ksize, ksize), np.uint8)
    out = cv2.morphologyEx(binimg, cv2.MORPH_OPEN, k)
    out = cv2.morphologyEx(out,    cv2.MORPH_CLOSE, k)
    return out

def segmentar_floculos_blanco(gray, mask_circle):
    """BLANCO = flóculo (OTSU sobre imagen con fondo aplanado + limpieza)."""
    blur = cv2.GaussianBlur(gray, (_odd(BLUR_KSIZE), _odd(BLUR_KSIZE)), 0)
    ker  = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (_odd(BG_OPEN_K), _odd(BG_OPEN_K)))
    bg   = cv2.morphologyEx(blur, cv2.MORPH_OPEN, ker)
    flat = cv2.subtract(blur, bg)  # resalta blancos locales
    flat = cv2.normalize(flat, None, 0, 255, cv2.NORM_MINMAX)
    _, m = cv2.threshold(flat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    m = cv2.bitwise_and(m, mask_circle)
    m = limpiar(m, OPEN_CLOSE)
    return m

def kernel_lineal(length, width, angle_deg):
    L = max(5, int(length) | 1)
    W = max(1, int(width)  | 1)
    k = np.zeros((L, L), np.uint8)
    cv2.line(k, (L//2, (L-W)//2), (L//2, (L+W)//2), 1, W)
    M = cv2.getRotationMatrix2D((L/2, L/2), angle_deg, 1.0)
    k = cv2.warpAffine(k, M, (L, L), flags=cv2.INTER_NEAREST)
    k[k>0] = 1
    return k

def realce_filamentos_multiescala(gray):
    """Devuelve respuesta máxima combinando tophat (claros) y/o blackhat (oscuros)."""
    # Quita iluminación de fondo
    bg = cv2.morphologyEx(gray, cv2.MORPH_OPEN,
                          cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (41, 41)))
    base = cv2.subtract(gray, bg)

    resp_max = np.zeros_like(base, np.uint8)
    use_bright = DARK_LINES_MODE in ("auto", "bright")
    use_dark   = DARK_LINES_MODE in ("auto", "dark")

    for L in LINE_LENGTHS:
        for W in LINE_WIDTHS:
            for ang in ANGLES_DEG:
                k = kernel_lineal(L, W, ang)
                if use_bright:
                    r = cv2.morphologyEx(base, cv2.MORPH_TOPHAT, k)
                    resp_max = np.maximum(resp_max, r)
                if use_dark:
                    r = cv2.morphologyEx(base, cv2.MORPH_BLACKHAT, k)
                    resp_max = np.maximum(resp_max, r)
    return cv2.normalize(resp_max, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

def reconstruccion_por_histeresis(resp, region, perc_hi, perc_lo):
    vals = resp[region > 0]
    if vals.size == 0: return np.zeros_like(resp)
    thr_hi = int(np.percentile(vals, perc_hi))
    thr_lo = int(np.percentile(vals, perc_lo))
    _, seeds   = cv2.threshold(resp, thr_hi, 255, cv2.THRESH_BINARY)
    _, support = cv2.threshold(resp, thr_lo, 255, cv2.THRESH_BINARY)
    seeds   = cv2.bitwise_and(seeds,   region)
    support = cv2.bitwise_and(support, region)
    rec = seeds.copy()
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    while True:
        new = cv2.bitwise_and(cv2.dilate(rec, k), support)
        if np.array_equal(new, rec): break
        rec = new
    return rec

def histeresis_auto(resp, region):
    if not AUTO_HYST:
        return reconstruccion_por_histeresis(resp, region, 90, 75)
    tot = max(1, np.count_nonzero(region))
    cands = []
    for hi in HYST_HI_CANDS:
        lo = max(60, hi - HYST_LO_DELTA)
        rec = reconstruccion_por_histeresis(resp, region, hi, lo)
        ra  = np.count_nonzero(rec)/tot
        # score: penaliza salir del rango y la distancia al target
        penalty = 0.0
        if ra < RA_FIL_OK_MIN: penalty += (RA_FIL_OK_MIN - ra)*10
        if ra > RA_FIL_OK_MAX: penalty += (ra - RA_FIL_OK_MAX)*10
        score = penalty + abs(ra - RA_FIL_TARGET)
        cands.append((score, ra, hi, lo, rec))
    cands.sort(key=lambda x:x[0])
    best = cands[0]
    print(f"[Fil] histeresis hi={best[2]} lo={best[3]} -> RA_fil_reg={best[1]*100:.2f}%")
    return best[4]

def limpiar_componentes(binimg, min_area):
    n, labels, stats, _ = cv2.connectedComponentsWithStats(binimg, connectivity=8)
    out = np.zeros_like(binimg)
    for i in range(1, n):
        if stats[i, cv2.CC_STAT_AREA] >= min_area:
            out[labels == i] = 255
    return out

def overlay_colores(bgr, mask_bg, mask_floc, mask_fil, alpha=0.6):
    color = np.zeros_like(bgr)
    color[mask_bg  > 0] = (200,150, 80)   # AZUL
    color[mask_floc> 0] = ( 80,220,120)   # VERDE
    color[mask_fil > 0] = (  0,165,255)   # NARANJO
    return cv2.addWeighted(color, alpha, bgr, 1-alpha, 0)

# ===== Métricas (sin normalizar por volumen) =====
def obtener_AT_AHD(mask_total):
    k = np.ones((2,2), np.uint8)
    mask_hd = cv2.morphologyEx(mask_total, cv2.MORPH_OPEN, k)
    mask_hd = cv2.morphologyEx(mask_hd, cv2.MORPH_OPEN, k)
    return mask_total, mask_hd

def contornos_validos(mask, min_area=MIN_FLOC_PIX):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    return [c for c in cnts if cv2.contourArea(c) >= min_area]

def medidas_por_floc(cnts, px_area_mm2, px_mm):
    res = []
    for i, c in enumerate(cnts, 1):
        area_px = cv2.contourArea(c)
        perim_px = cv2.arcLength(c, True)
        M = cv2.moments(c)
        if M['m00'] == 0: continue
        cx = M['m10']/M['m00']; cy = M['m01']/M['m00']
        pts = c.reshape(-1,2)
        d = np.sqrt((pts[:,0]-cx)**2 + (pts[:,1]-cy)**2)
        rmin_px = np.min(d) if d.size>0 else 0.0
        rmax_px = np.max(d) if d.size>0 else 0.0

        area_mm2 = area_px * px_area_mm2
        perim_mm = perim_px * px_mm
        rmin_mm  = rmin_px * px_mm
        rmax_mm  = rmax_px * px_mm
        AR = 4.0*(rmax_mm - rmin_mm)/(np.pi*rmin_mm) + 1.0 if rmin_mm>0 else np.nan
        RN = (perim_mm**2)/(4.0*np.pi*area_mm2) if area_mm2>0 else np.nan

        res.append(dict(id=i, area_px=area_px, area_mm2=area_mm2,
                        perim_px=perim_px, perim_mm=perim_mm,
                        rmin_mm=rmin_mm, rmax_mm=rmax_mm, AR=AR, RN=RN,
                        cx_px=cx, cy_px=cy))
    return res

def fractal_dimension_loglog(rows):
    xs, ys = [], []
    for r in rows:
        if r['area_mm2']>0 and r['perim_mm']>0:
            xs.append(np.log(r['perim_mm'])); ys.append(np.log(r['area_mm2']))
    if len(xs) >= 2:
        m, _ = np.polyfit(xs, ys, 1)
        return float(m)
    return np.nan

def fil_length_mm(mask_fil, px_mm):
    method = "area_over_width"
    try:
        from skimage.morphology import skeletonize
        skel = skeletonize((mask_fil>0).astype(bool))
        length_px = int(np.count_nonzero(skel))
        method = "skeletonize"
    except Exception:
        length_px = int(np.count_nonzero(mask_fil) / max(LINE_WIDTHS[0],1))
    return length_px * px_mm, method


# --------- PROCESO (1 imagen) ---------
img_path = ruta_base / nombre_imagen
bgr  = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
if bgr is None: raise SystemExit(f"❌ No pude leer {img_path}")
gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

# Campo de visión
mask_circle = mascara_campo(gray, RADIO_REL_CAMPO)

# 1) Flóculos (BLANCO = masa)
mask_floc_raw = segmentar_floculos_blanco(gray, mask_circle)
mask_floc = mask_floc_raw.copy()

# 2) Filamentos: realce multi-escala + histeresis auto
resp = realce_filamentos_multiescala(gray)

region_fil = mask_circle.copy()
if SOLO_SOBRESAL:
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (_odd(2*MARGEN_PX+1), _odd(2*MARGEN_PX+1)))
    region_fil = cv2.bitwise_and(region_fil, cv2.bitwise_not(cv2.dilate(mask_floc, k)))
if np.count_nonzero(region_fil) < 0.01 * np.count_nonzero(mask_circle):
    region_fil = mask_circle.copy()  # fallback

mask_fil = histeresis_auto(resp, region_fil)
mask_fil = limpiar_componentes(mask_fil, MIN_FIL_PIX)

# 3) Restar filamentos a la masa (margen) para evitar solapes
if QUITAR_DE_MASA_MARGEN > 0:
    kd = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (_odd(2*QUITAR_DE_MASA_MARGEN+1), _odd(2*QUITAR_DE_MASA_MARGEN+1)))
    fil_dil = cv2.dilate(mask_fil, kd)
    mask_floc = cv2.bitwise_and(mask_floc, cv2.bitwise_not(fil_dil))

# 4) Fondo
mask_bg = mask_circle.copy()
mask_bg[mask_floc > 0] = 0
mask_bg[mask_fil  > 0] = 0

# ===== SALIDAS DE IMAGEN =====
overlay = overlay_colores(bgr, mask_bg, mask_floc, mask_fil, alpha=0.60)
out_overlay = img_path.with_name(f"{img_path.stem}_overlay.png");   cv2.imwrite(str(out_overlay), overlay)

h, w = gray.shape
tile_w, tile_h = w//2, h//2
to_bgr = lambda im: cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
rsz    = lambda im: cv2.resize(to_bgr(im), (tile_w, tile_h), interpolation=cv2.INTER_AREA)
panel  = np.vstack([ np.hstack([rsz(gray), rsz(resp)]),
                     np.hstack([rsz(mask_floc), rsz(mask_fil)]) ])
out_panel = img_path.with_name(f"{img_path.stem}_masks.png");       cv2.imwrite(str(out_panel), panel)

out_mask_bg   = img_path.with_name(f"{img_path.stem}_mask_bg.png");   cv2.imwrite(str(out_mask_bg),   mask_bg)
out_mask_floc = img_path.with_name(f"{img_path.stem}_mask_floc.png"); cv2.imwrite(str(out_mask_floc), mask_floc)
out_mask_fil  = img_path.with_name(f"{img_path.stem}_mask_fil.png");  cv2.imwrite(str(out_mask_fil),  mask_fil)

# ====== MÉTRICAS Y EXCEL (sin /mL) ======
px_mm       = float(PIXEL_MM)
px_area_mm2 = px_mm**2

pix_cir = int(np.count_nonzero(mask_circle > 0))
effective_area_mm2 = pix_cir * px_area_mm2

cnt_floc = int(np.count_nonzero((mask_floc>0)))
cnt_fil  = int(np.count_nonzero((mask_fil>0)))
cnt_bg   = int(np.count_nonzero((mask_bg>0)))
tot      = cnt_floc + cnt_fil + cnt_bg

RA_floc = 100.0*cnt_floc/tot if tot>0 else 0.0
RA_fil  = 100.0*cnt_fil /tot if tot>0 else 0.0
RA_bg   = 100.0*cnt_bg  /tot if tot>0 else 0.0

mask_AT, mask_AHD = obtener_AT_AHD(mask_floc_raw)
AT_mm2  = np.count_nonzero(mask_AT>0)  * px_area_mm2
AHD_mm2 = np.count_nonzero(mask_AHD>0) * px_area_mm2
AHD_sobre_AT = (AHD_mm2/AT_mm2) if AT_mm2>0 else np.nan

DeqT  = 2.0*math.sqrt(AT_mm2/np.pi)  if AT_mm2>0  else np.nan
DeqHD = 2.0*math.sqrt(AHD_mm2/np.pi) if AHD_mm2>0 else np.nan

cnts_total = contornos_validos(mask_AT, MIN_FLOC_PIX)
rows = medidas_por_floc(cnts_total, px_area_mm2, px_mm)
AR_mean = float(np.nanmean([r['AR'] for r in rows])) if rows else np.nan
RN_mean = float(np.nanmean([r['RN'] for r in rows])) if rows else np.nan
FD_val  = fractal_dimension_loglog(rows) if rows else np.nan

fil_length_mm_val, fil_len_method = fil_length_mm(mask_fil, px_mm)

# ---- Exportar a Excel ----
try:
    import pandas as pd
except ImportError:
    raise SystemExit("Instala pandas y openpyxl:  pip install pandas openpyxl")

out_xlsx = img_path.with_name(f"{img_path.stem}_medidas.xlsx")

df_resumen = pd.DataFrame([{
    "Imagen": img_path.name,
    "Ancho (px)": w, "Alto (px)": h, "Diám. campo (px)": DIAM_PX_CAMPO,
    "mm/px": px_mm, "Área efectiva (mm²)": effective_area_mm2,
    "Fondo (px)": cnt_bg, "Flóculos (px)": cnt_floc, "Filamentos (px)": cnt_fil,
    "RA fondo (%)": RA_bg, "RA flóculos (%)": RA_floc, "RA filamentos (%)": RA_fil,
    "AT (mm²)": AT_mm2, "AHD (mm²)": AHD_mm2, "AHD/AT": AHD_sobre_AT,
    "DeqT (mm)": DeqT, "DeqHD (mm)": DeqHD,
    "AR̅": AR_mean, "RN̅": RN_mean, "FD": FD_val,
    "Long. filamentos (mm)": fil_length_mm_val
}])
df_resumen = df_resumen[[
    "Imagen","Ancho (px)","Alto (px)","Diám. campo (px)","mm/px","Área efectiva (mm²)",
    "Fondo (px)","Flóculos (px)","Filamentos (px)",
    "RA fondo (%)","RA flóculos (%)","RA filamentos (%)",
    "AT (mm²)","AHD (mm²)","AHD/AT","DeqT (mm)","DeqHD (mm)",
    "AR̅","RN̅","FD","Long. filamentos (mm)"
]]

df_flocs = pd.DataFrame(rows) if rows else pd.DataFrame(columns=[
    "id","area_px","area_mm2","perim_px","perim_mm","rmin_mm","rmax_mm","AR","RN","cx_px","cy_px"
])

tot_safe = max(1, tot)
df_pix = pd.DataFrame([{
    "imagen": img_path.name, "total_region": tot_safe, "bg_px": cnt_bg, "floc_px": cnt_floc, "fil_px": cnt_fil
}])

df_cfg = pd.DataFrame([{
    "PIXEL_MM": px_mm, "px_area_mm2": px_area_mm2,
    "DIAM_PX_CAMPO": DIAM_PX_CAMPO, "RADIO_REL_CAMPO": RADIO_REL_CAMPO,
    "EFFECTIVE_AREA_MM2": effective_area_mm2,
    "DARK_LINES_MODE": DARK_LINES_MODE, "LINE_LENGTHS": str(LINE_LENGTHS), "LINE_WIDTHS": str(LINE_WIDTHS),
    "ANGLES_DEG": str(ANGLES_DEG),
    "AUTO_HYST": AUTO_HYST, "HYST_HI_CANDS": str(HYST_HI_CANDS),
    "HYST_LO_DELTA": HYST_LO_DELTA, "RA_FIL_TARGET": RA_FIL_TARGET,
    "RA_FIL_OK_MIN": RA_FIL_OK_MIN, "RA_FIL_OK_MAX": RA_FIL_OK_MAX,
    "SOLO_SOBRESAL": SOLO_SOBRESAL, "MARGEN_PX": MARGEN_PX,
    "QUITAR_DE_MASA_MARGEN": QUITAR_DE_MASA_MARGEN,
    "BG_OPEN_K": BG_OPEN_K, "BLUR_KSIZE": BLUR_KSIZE, "OPEN_CLOSE": OPEN_CLOSE,
    "MIN_FLOC_PIX": MIN_FLOC_PIX,
    "fil_length_method": fil_len_method
}])

with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    df_resumen.to_excel(writer, index=False, sheet_name="Resumen")
    df_flocs.to_excel(writer, index=False, sheet_name="Flocs_por_objeto")
    df_pix.to_excel(writer, index=False, sheet_name="Pixeles_label")
    df_cfg.to_excel(writer, index=False, sheet_name="Config")

print("✅ Listo")
print("Overlay:", out_overlay)
print("Panel  :", out_panel)
print("Masks -> BG:", out_mask_bg, " Floc:", out_mask_floc, " Fil:", out_mask_fil)
print("Excel  :", out_xlsx)


[Fil] histeresis hi=97 lo=82 -> RA_fil_reg=14.54%
✅ Listo
Overlay: C:\Users\PC\Desktop\indice de jenkins\filtro erosion\2_semana\procesadas_erosion\2a_overlay.png
Panel  : C:\Users\PC\Desktop\indice de jenkins\filtro erosion\2_semana\procesadas_erosion\2a_masks.png
Masks -> BG: C:\Users\PC\Desktop\indice de jenkins\filtro erosion\2_semana\procesadas_erosion\2a_mask_bg.png  Floc: C:\Users\PC\Desktop\indice de jenkins\filtro erosion\2_semana\procesadas_erosion\2a_mask_floc.png  Fil: C:\Users\PC\Desktop\indice de jenkins\filtro erosion\2_semana\procesadas_erosion\2a_mask_fil.png
Excel  : C:\Users\PC\Desktop\indice de jenkins\filtro erosion\2_semana\procesadas_erosion\2a_medidas.xlsx
